Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [ ]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [ ]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [ ]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [ ]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
  tf.set_random_seed(1234)

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1, seed=12345), name='layer1_weights')
  layer1_biases = tf.Variable(tf.zeros([depth]), name='layer1_biases')
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1, seed=12347), name='layer2_weights')
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]), name='layer2_biases')
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1, seed=12349), name='layer3_weights')
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]), name='layer3_biases')
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1, seed=12351), name='layer4_weights')
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='layer4_biases')
  
  # Model.
  def model(data):
###    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME', name='conv1')
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME', name='conv1')
    max_pool_1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME',name='max_pool_1')
###    hidden1 = tf.nn.relu(conv1 + layer1_biases, name='hidden_relu_1')
    hidden1 = tf.nn.relu(max_pool_1 + layer1_biases, name='hidden_relu_1')
###    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding='SAME', name='conv2')
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 1, 1, 1], padding='SAME', name='conv2')
    max_pool_2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME',name='max_pool_2')
###    hidden2 = tf.nn.relu(conv2 + layer2_biases, name='hidden_relu_2')
    hidden2 = tf.nn.relu(max_pool_2 + layer2_biases, name='hidden_relu_2')
    shape = hidden2.get_shape().as_list()
    reshape = tf.reshape(hidden2, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases, name='hidden_relu_3')
    return tf.matmul(hidden3, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  with tf.name_scope('loss'):
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  with tf.name_scope('prediction'):
    train_prediction = tf.nn.softmax(logits, name='softmax_train_prediction')
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset), name='softmax_valid_prediction')
    test_prediction = tf.nn.softmax(model(tf_test_dataset), name='softmax_test_prediction')

num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  train_writer = tf.summary.FileWriter('/tmp/tensorboard/cnn/run9', session.graph)
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Performance before changes:  
Minibatch loss at step 950: 0.611567  
Minibatch accuracy: 81.2%  
Validation accuracy: 82.7%  
Minibatch loss at step 1000: 0.406159  
Minibatch accuracy: 87.5%  
Validation accuracy: 81.9%  
Test accuracy: 89.3%  

Performance after changes (adding max-pooling):
Minibatch loss at step 950: 0.488780  
Minibatch accuracy: 81.2%  
Validation accuracy: 84.2%  
Minibatch loss at step 1000: 0.375930  
Minibatch accuracy: 93.8%  
Validation accuracy: 84.0%  
Test accuracy: 90.7%  


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [ ]:
#
#
# CNN with dropouts and learning rate decay
#
#
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

###graph = tf.Graph()
tf.reset_default_graph()
###graph = tf.Graph()
session = tf.Session()
with session.as_default():
  tf.set_random_seed(1234)

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

# Set up dropout probability placeholder - only apply during training.
  keep_prob_val = tf.placeholder(tf.float32,shape=())  

  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1, seed=12345), name='layer1_weights')
  layer1_biases = tf.Variable(tf.zeros([depth]), name='layer1_biases')
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1, seed=12347), name='layer2_weights')
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]), name='layer2_biases')
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1, seed=12349), name='layer3_weights')
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]), name='layer3_biases')
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1, seed=12351), name='layer4_weights')
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]), name='layer4_biases')
  
  # Model.
  def model(data):
###    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME', name='conv1')
    conv1 = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME', name='conv1')
    max_pool_1 = tf.nn.max_pool(conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME',name='max_pool_1')
###    hidden1 = tf.nn.relu(conv1 + layer1_biases, name='hidden_relu_1')
    hidden1 = tf.nn.relu(max_pool_1 + layer1_biases, name='hidden_relu_1')
###    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 2, 2, 1], padding='SAME', name='conv2')
    conv2 = tf.nn.conv2d(hidden1, layer2_weights, [1, 1, 1, 1], padding='SAME', name='conv2')
    max_pool_2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME',name='max_pool_2')
###    hidden2 = tf.nn.relu(conv2 + layer2_biases, name='hidden_relu_2')
    hidden_relu_2 = tf.nn.relu(max_pool_2 + layer2_biases, name='hidden_relu_2')
    dropout_1 = tf.nn.dropout(hidden_relu_2, keep_prob_val, seed=123477, name='dropout_1')
    shape = dropout_1.get_shape().as_list()
    reshape = tf.reshape(dropout_1, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden3 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases, name='hidden_relu_3')
    return tf.matmul(hidden3, layer4_weights) + layer4_biases, dropout_1, hidden_relu_2
  
  # Training computation.
  logits, dropout_test, hidden_test = model(tf_train_dataset)
  with tf.name_scope('loss'):
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  starter_learning_rate = 0.05
  decay_steps = 1000
  decay_rate=0.96
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           decay_steps, decay_rate, staircase=False)

  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
###  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  with tf.name_scope('prediction'):
    train_prediction = tf.nn.softmax(logits, name='softmax_train_prediction')
    vp, dropout_test_vp, hidden_test_vp = model(tf_valid_dataset)
    tp, dropout_test_tp, hidden_test_tp = model(tf_test_dataset)
    valid_prediction = tf.nn.softmax(vp, name='softmax_valid_prediction')
    test_prediction = tf.nn.softmax(tp, name='softmax_test_prediction')

#
# Now do training.
#
  num_steps = 1001

#with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  train_writer = tf.summary.FileWriter('/tmp/tensorboard/cnn_drop/run4', session.graph)
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, 
                 keep_prob_val : 0.5 }
    _, l, predictions, dropout_test_res, hidden_test_res, learning_rate_res = session.run(
      [optimizer, loss, train_prediction, dropout_test, hidden_test, learning_rate], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('hidden_test_res[0,0,0,0], hidden_test_res[0,0,0,1] = ', hidden_test_res[0,0,0,0], hidden_test_res[0,0,0,1])
      print('dropout_test_res[0,0,0,0], dropout_test_res[0,0,0,1] = ', dropout_test_res[0,0,0,0], dropout_test_res[0,0,0,1])
      print('learning_rate_res = ', learning_rate_res)
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
###      dropout = session.run(dropout_1,feed_dict=feed_dict)
###      print('dropout_1[0,0,0,0], dropout_1[0,0,0,1] = ', dropout[0,0,0,0], dropout[0,0,0,1] )
###      print('Validation accuracy: %.1f%%' % accuracy(
###        valid_prediction.eval(), valid_labels))
      feed_dict = { keep_prob_val : 1.0 }
      predictions = session.run(valid_prediction, feed_dict=feed_dict)
      print('Validation accuracy: %.1f%%' % accuracy(predictions, valid_labels))
    
  # Test predictions.
  feed_dict = { keep_prob_val : 1.0 }
  predictions = session.run(test_prediction, feed_dict=feed_dict)
  print('Test accuracy: %.1f%%' % accuracy(predictions, test_labels))

Before modifications (should match results from max-pool problem 1)  
Minibatch loss at step 950: 0.488780  
Minibatch accuracy: 81.2%  
Validation accuracy: 84.2%  
Minibatch loss at step 1000: 0.375930  
Minibatch accuracy: 93.8%  
Validation accuracy: 84.0%  
Test accuracy: 90.7%  

Results from max-pool problem 1:  
Performance after changes (adding max-pooling):  
Minibatch loss at step 950: 0.488780  
Minibatch accuracy: 81.2%  
Validation accuracy: 84.2%  
Minibatch loss at step 1000: 0.375930  
Minibatch accuracy: 93.8%  
Validation accuracy: 84.0%  
Test accuracy: 90.7%

Results after modifications (adding drop outs - 50%):  
Minibatch loss at step 950: 0.523877  
Minibatch accuracy: 87.5%  
Validation accuracy: 84.0%  
Minibatch loss at step 1000: 0.334872  
Minibatch accuracy: 87.5%  
Validation accuracy: 83.8%  
Test accuracy: 90.3%  

Results after modifications (adding drop outs - 50% + learning rate decay):
Minibatch loss at step 950: 0.414466  
Minibatch accuracy: 87.5%  
Validation accuracy: 83.9%  
Minibatch loss at step 1000: 0.471627  
Minibatch accuracy: 81.2%  
Validation accuracy: 84.0%  
Test accuracy: 90.8%  